#### This notebook gives an overview of Harrow Hassidim Lloyd algorithm.
Source: https://qiskit.org/textbook/ch-applications/hhl_tutorial.html

A very excellent lecture from Prof. Petteri Kaski on this can be found in: https://aalto.cloud.panopto.eu/Panopto/Pages/Viewer.aspx?id=951442f6-7ce0-451f-b2b6-ad5d010a0cd0 (Requires login and permission to access the video)

#### Problem setup

We have the following system:

$$
x_1 - \frac{x_2}{3} = 1 \\
- \frac{x_1}{3} + x_2 = 0
$$

where $A = \begin{bmatrix} 1 & -1/3 \\ - 1/3 & 1 \end{bmatrix} \in \mathbb{C}^{N \times N}$, $\vec{b} = \begin{bmatrix}1 & 0\end{bmatrix}^T \in \mathbb{C}^N$, and $\vec{x} = \begin{bmatrix}x_1 & x_2\end{bmatrix}^T \in \mathbb{C}^N$. So, we can write the system in matrix form

$$
A \vec{x} = \vec{b}.
$$

- s-sparse: If the matrix $A$ contains at most $s$ non-zero entries per row(column?), then  the system of linear equations is called s-sparse.
- A s-parse system with size $N$ requires $\mathcal{O}(N s \kappa \log(1/\epsilon))$ running time complexity, where $\kappa$ is the condition number of the system and $\epsilon$ is the accuracy of the approximation.
- HHL algorithm estimates a function of the solution with running time complexity of $\mathcal{O}( \log(N) s^2 \kappa^2 /\epsilon)$.

#### Why sparse solution?

- We are interested in solving the system for large N. Say, $N = 2^{100}$. 
- A is quite large but s-sparse, where $ s <<< N$.
- In this large system, we are either interested in getting the $k^{th}$ equation which has coefficients, $\{a_{kj}\}$, where $j \leq s$.

In reality, we cannot solve for the full solutions of such system with so large N. What we can do is that we try to get some information, for example what is the $j^{th}$ component of the solution or the sum of the solution vector.

#### Solving with Quantum computing
If we want that quantum computers can solve such problem, then we need to encode the problem in quantum language.
We normalize and map the vectors to their respective quatum states and the problem becomes
$$
 A \lvert x \rangle = \lvert b \rangle.
$$

If A is a Hermitian matrix, then it can be decomposed as
$$
A = \sum_{j=0}^{N-1} \lambda_j \lvert u_j \rangle \langle u_j \rvert.
$$
The inverse of A can be written as
$$
A^{-1} = \sum_{j=0}^{N-1} \lambda_j^{-1} \lvert u_j \rangle \langle u_j \rvert.
$$


The right hand of the equation can also be written as the eigenbasis of A as
$$
\lvert b \rangle = \sum_{j=0}^{N-1} b_j \lvert u_j \rangle
$$


The goal of the HHL algorithm is to exit the algorithm with the readout register in the state

$$
\lvert x \rangle = A^{-1} \lvert b \rangle \\
= \sum_{j=0}^{N-1} \lambda_j^{-1} b_j  \lvert u_j \rangle
$$

### Qiskit implementation
First, we try naive solutions using numpy and Qiskit. 

In [1]:
import numpy as np
A = np.array([
    [1, -1/3],
    [-1/3, 1]
])
b = np.array([1, 0])

#### solving with numpy package

In [2]:
x_numpy_sol = np.linalg.solve(A, b)
x_numpy_sol

array([1.125, 0.375])

In [3]:
np.linalg.norm(x_numpy_sol)

1.1858541225631423

Qiskit also has a numpy solver. This solver has the following properties:
- state: the circuit prepares the solution or the solution as a vector
- euclidean_norm: the eucledian norm if  the algorithm knows how to calculate it.
- observable: the list of calculated observables
- circuit_results: the observable results from the list of circuits.

Let's try it.

In [4]:
from qiskit.algorithms.linear_solvers.numpy_linear_solver import NumPyLinearSolver

In [5]:
x_classical_sol = NumPyLinearSolver().solve(A, b/np.linalg.norm(b))
x_classical_sol.state

array([1.125, 0.375])

In [6]:
print(x_classical_sol.state)

[1.125 0.375]


In [7]:
x_classical_sol.euclidean_norm

1.1858541225631423

In [8]:
print('error: ', np.linalg.norm(x_numpy_sol) - x_classical_sol.euclidean_norm)

error:  0.0


#### solving with Qiskit HHL solver

In [9]:
from qiskit.algorithms.linear_solvers.hhl import HHL

In [10]:
x_naive_hhl_sol = HHL().solve(A, b)
print(x_naive_hhl_sol.state)

      ┌───────────┐┌──────┐        ┌─────────┐
  q4: ┤ circuit-7 ├┤3     ├────────┤3        ├
      └───────────┘│      │┌──────┐│         │
q5_0: ─────────────┤0     ├┤2     ├┤0        ├
                   │  QPE ││      ││  QPE_dg │
q5_1: ─────────────┤1     ├┤1     ├┤1        ├
                   │      ││  1/x ││         │
q5_2: ─────────────┤2     ├┤0     ├┤2        ├
                   └──────┘│      │└─────────┘
  q6: ─────────────────────┤3     ├───────────
                           └──────┘           


As you can see, the quantum solution is not straightforward like numpy solver. There are three registers used to implement HHL algorithm (we will explain details soon). One of them is the auxiliary qubit, where we can measure the probability of 1 using the squared norm. This is actually the euclidian norm for the solution. Therefore, we can compute,

In [11]:
x_naive_hhl_sol.euclidean_norm

1.1858541225631374

In [12]:
print('error: ', x_naive_hhl_sol.euclidean_norm - x_classical_sol.euclidean_norm)

error:  -4.884981308350689e-15


The results are not same but quite close.

Now, we are also interested to compare whether the state vectors are same or not. Typically, we cannot obtain the full solution vector from the quantum algorithm. However, we can only approximate the solution vector and compare if that is good enough.

In [13]:
from qiskit.quantum_info import Statevector


In [14]:
x_naive = Statevector(x_naive_hhl_sol.state).data

In [15]:
x_naive.shape

(32,)

In [16]:
x_naive = np.array([x_naive[8], x_naive[9]])
x_naive

array([1.48829897e-16-1.68137822e-16j, 1.09592835e-16+7.51955611e-16j])

Here, we can see that components are mixed of real and complex number. Our solution should be real, so

In [17]:
x_real = np.real(x_naive)
x_real

array([1.48829897e-16, 1.09592835e-16])

Hey! This is not what we are expecting what's wrong!
The solution is affected by constants coming from different parts of the circuits. And, we can recover the solution by multiplying these normalized vectors by the respective Euclidean norm.

In [18]:
x_real*x_naive_hhl_sol.euclidean_norm/np.linalg.norm(x_real)

array([0.9548972, 0.703151 ])